## 瑜珈分數計算

- 及時計算使用者目前瑜珈動作的分數
- 主程式: ScoreCalculator.py


### 1. python class 方置區

此區放置之後回移植到 android studio 的 python 檔案
在完成研究後，記得把程式碼複製上去

- ScoreCalculator.py

In [1]:
from AngleCalculator import calculate_angle, calculate_angle_in_andriodStudio
from yogaFileGetter import get_sample_angle_dict, get_std_angle_dict, get_weight_angle_dict


class ScoreCalculator:
    def __init__(self, pose_name):
        self.pose_name = pose_name

        self.average_angle_dict = get_sample_angle_dict(pose_name)
        self.std_angle_dict = get_std_angle_dict(pose_name)
        self.weight = get_weight_angle_dict(pose_name)

        self.score_dict = {}

    def calculate_score(self, point3d, is_android_studio):
        if is_android_studio:
            input_angle = calculate_angle_in_andriodStudio(self.pose_name, point3d)
        else:
            input_angle = calculate_angle(self.pose_name, point3d)

        self.score_dict = {}
        for joint, input_value in input_angle.items():
            average_value = self.average_angle_dict.get(joint, None)
            std_value = self.std_angle_dict.get(joint, None)

            if average_value is not None and std_value is not None:
                diff = abs(input_value - average_value)

                if diff <= 2 * std_value:
                    self.score_dict[joint] = 100
                elif diff <= 3 * std_value:
                    self.score_dict[joint] = 50
                else:
                    self.score_dict[joint] = 0

        # 使用字典生成式計算差異
        difference_dict = {joint: input_angle[joint] - self.average_angle_dict[joint] for joint in input_angle if
                           joint in self.average_angle_dict}
        difference_dict = {joint: round(value, 1) for joint, value in difference_dict.items()}

        # 打印結果
        print("偵測角度", input_angle)
        print("平均角度", self.average_angle_dict)
        print("關節差距", difference_dict)
        # 使用字典生成式將所有元素乘以 2
        full_score_range_dict = {joint: value * 2 for joint, value in self.std_angle_dict.items()}
        print("滿分區間", full_score_range_dict)
        print("關節得分", self.score_dict)

        # 計算總和
        sum_score = 0
        max_score = 0
        for joint, score in self.score_dict.items():
            weight = self.weight.get(joint, None)

            sum_score += weight * score
            max_score += weight

        # 計算平均值
        average_value = sum_score / max_score

        return int(average_value)

## 2. 研究區

此區放置不會移植到 android 的檔案

### 測試瑜珈分數計算器

In [2]:

from AngleCalculator import calculate_angle
from develop_tool import draw_skeleton, get_image_files, get_Mediapipe_point
import cv2

# 顯示有分數 + 骨架的圖片
def display(image, name, pose, calculator):
    point2d, point3d = get_Mediapipe_point(image)
    if point2d is not None :
        score = calculator.calculate_score(point3d, False)
        print(score)

        cv2.putText(image, "Score:" + str(score), (10, 100), cv2.FONT_HERSHEY_PLAIN, 3, (0, 0, 255), 5)
        image2 = image

        # cv2.imshow(name, image)
        draw_skeleton(image2, point2d)
        cv2.imshow(name + "2", image2)

# 顯示影片
def show_video(video_path, pose, calculator):
    cap = cv2.VideoCapture(video_path)

    while True:
        ret, frame = cap.read()
        if not ret:
            print("Video end")
            break

        display(frame, "video", pose, calculator)

        if cv2.waitKey(1) & 0xFF == ord('q'):
            break

    cap.release()
    cv2.destroyAllWindows()
    
    
# 顯示整個資料夾的圖片
def show_image_list(image_path, pose, calculator):
    image_files = get_image_files(image_path)

    for name in image_files:
        print(name)
        image = cv2.imread(name, cv2.IMREAD_UNCHANGED)

        display(image, name, pose, calculator)

    # 按下任意鍵則關閉所有視窗
    cv2.waitKey(0)
    cv2.destroyAllWindows()

# 顯示單張圖片
def show_image(image_path, pose, calculator):
    image = cv2.imread(image_path, cv2.IMREAD_UNCHANGED)

    display(image, image_path, pose, calculator)

    # 按下任意鍵則關閉所有視窗
    cv2.waitKey(0)
    cv2.destroyAllWindows()





In [3]:
pose_list = ["Tree Style", "Warrior2 Style", "Plank", "Reverse Plank", "Child's pose", "Downward dog", "Low Lunge", "Seated Forward Bend", "Bridge pose", "Pyramid pose"]

pose = 'Tree Style'
calculator = ScoreCalculator(pose)

# 顯示影片
# video_path = "../data/video/棒式教學.mp4"
# show_video(video_path , pose, calculator) 

# 顯示單張圖片
# image_path = "data/image/LowLunge/cobra2.png"
# show_image(image_path , pose, calculator) 

# 顯示整個資料夾的圖片
folder_path = "../data/image/" + pose
show_image_list(folder_path, pose, calculator) 

../data/image/Tree Style\0-1.png
偵測角度 {'LEFT_SHOULDER': 34.7, 'RIGHT_SHOULDER': 12.2, 'LEFT_ELBOW': 82.8, 'RIGHT_ELBOW': 91.4, 'LEFT_HIP': 107.3, 'RIGHT_HIP': 137.6, 'LEFT_KNEE': 136.3, 'RIGHT_KNEE': 50.0}
平均角度 {'LEFT_SHOULDER': 165.88890864673917, 'RIGHT_SHOULDER': 153.20828845486872, 'LEFT_ELBOW': 153.61120642376665, 'RIGHT_ELBOW': 163.3014742620245, 'LEFT_HIP': 97.64611072133951, 'RIGHT_HIP': 130.5388741995487, 'LEFT_KNEE': 145.84625672299853, 'RIGHT_KNEE': 56.218335997632025}
關節差距 {'LEFT_SHOULDER': -131.2, 'RIGHT_SHOULDER': -141.0, 'LEFT_ELBOW': -70.8, 'RIGHT_ELBOW': -71.9, 'LEFT_HIP': 9.7, 'RIGHT_HIP': 7.1, 'LEFT_KNEE': -9.5, 'RIGHT_KNEE': -6.2}
滿分區間 {'LEFT_SHOULDER': 8.2, 'RIGHT_SHOULDER': 11.6, 'LEFT_ELBOW': 10.0, 'RIGHT_ELBOW': 9.8, 'LEFT_HIP': 9.2, 'RIGHT_HIP': 7.0, 'LEFT_KNEE': 8.0, 'RIGHT_KNEE': 11.8}
關節得分 {'LEFT_SHOULDER': 0, 'RIGHT_SHOULDER': 0, 'LEFT_ELBOW': 0, 'RIGHT_ELBOW': 0, 'LEFT_HIP': 50, 'RIGHT_HIP': 50, 'LEFT_KNEE': 50, 'RIGHT_KNEE': 100}
31
../data/image/Tree Sty